<a href="https://colab.research.google.com/github/srujanreddyj/Data-Engineering-concepts/blob/master/udacity_capstone_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Udacity Data Engineering/airbnb
%pwd

/content/drive/MyDrive/Udacity Data Engineering/airbnb


'/content/drive/MyDrive/Udacity Data Engineering/airbnb'

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/drive/MyDrive/Udacity Data Engineering/airbnb/spark-2.4.5-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/drive/MyDrive/Udacity Data Engineering/airbnb/spark-3.1.1-bin-hadoop2.7"


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc




<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

In [ ]:
spark = SparkSession.builder.appName("de_project")\
.config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true")\
.getOrCreate()

spark

In [ ]:
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            #column primary key
            key = rows['listing_id']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         

In [ ]:
# Decide the two file paths according to your
# computer system
csvFilePath_austin = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/LA_reviews.csv'
jsonFilePath_austin = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/austin_reviews.json'

csvFilePath_la = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/LA/la_reviews.csv'
jsonFilePath_la = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/LA/la_reviews.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath, 'listing_id')

In [ ]:
csv_austin_listing = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/austin_listings.csv'
json_asutin_listing = r'/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/austin_listing.json'
make_json(csv_austin_listing, json_asutin_listing, 'id')


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DecimalType, DoubleType, IntegerType, DateType,FloatType
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import *
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

#### LISTING TABLE

In [ ]:
customSchema = StructType([
    T.StructField("id", T.IntegerType(), True),
    T.StructField("listing_url", T.StringType(), True),
    T.StructField("scrape_id", T.DecimalType(20), True),
    T.StructField("last_scraped", T.DateType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("description", T.StringType(), True),
    T.StructField("neighborhood_overview", T.StringType(), True),
    T.StructField("picture_url", T.StringType(), True),
    T.StructField("host_id", T.IntegerType(), True),
    T.StructField("host_url", T.StringType(), True),
    T.StructField("host_name", T.StringType(), True),
    T.StructField("host_since", T.DateType(), True),
    T.StructField("host_location", T.StringType(), True),
    T.StructField("host_about", T.StringType(), True),
    T.StructField("host_response_time", T.StringType(), True),
    T.StructField("host_response_rate", T.StringType(), True),
    T.StructField("host_acceptance_rate", T.StringType(), True),
    T.StructField("host_is_superhost", T.StringType(), True),
    T.StructField("host_thumbnail_url", T.StringType(), True),
    T.StructField("host_picture_url", T.StringType(), True),
    T.StructField("host_neighbourhood", T.StringType(), True),
    T.StructField("host_listings_count", T.DoubleType(), True),
    T.StructField("host_total_listings_count", T.DoubleType(), True),
    T.StructField("host_verifications", T.StringType(), True),
    T.StructField("host_has_profile_pic", T.StringType(), True),
    T.StructField("host_identity_verified", T.StringType(), True),
    T.StructField("neighbourhood", T.StringType(), True),
    T.StructField("neighbourhood_cleansed", T.IntegerType(), True),
    T.StructField("neighbourhood_group_cleansed", T.StringType(), True),
    T.StructField("latitude", T.DoubleType(), True),
    T.StructField("longitude", T.DoubleType(), True),
    T.StructField("property_type", T.StringType(), True),
    T.StructField("room_type", T.StringType(), True),
    T.StructField("accommodates", T.IntegerType(), True),
    T.StructField("bathrooms", T.DoubleType(), True),
    T.StructField("bathrooms_text", T.StringType(), True),
    T.StructField("bedrooms", T.DoubleType(), True),
    T.StructField("beds", T.DoubleType(), True),
    T.StructField("amenities", T.StringType(), True),
    T.StructField("price", T.StringType(), True),
    T.StructField("minimum_nights", T.IntegerType(), True),
    T.StructField("maximum_nights", T.IntegerType(), True),
    T.StructField("minimum_minimum_nights", T.DoubleType(), True),
    T.StructField("maximum_minimum_nights", T.DoubleType(), True),
    T.StructField("minimum_maximum_nights", T.DoubleType(), True),
    T.StructField("maximum_maximum_nights", T.DoubleType(), True),
    T.StructField("minimum_nights_avg_ntm", T.DoubleType(), True),
    T.StructField("maximum_nights_avg_ntm", T.DoubleType(), True),
    T.StructField("calendar_updated", T.DateType(), True),
    T.StructField("has_availability", T.StringType(), True),
    T.StructField("availability_30", T.DoubleType(), True),
    T.StructField("availability_60", T.DoubleType(), True),
    T.StructField("availability_90", T.DoubleType(), True),
    T.StructField("availability_365", T.DoubleType(), True),
    T.StructField("calendar_last_scraped", T.DateType(), True),
    T.StructField("number_of_reviews", T.IntegerType(), True),
    T.StructField("number_of_reviews_ltm", T.DoubleType(), True),
    T.StructField("number_of_reviews_l30d", T.DoubleType(), True),
    T.StructField("first_review", T.DateType(), True),
    T.StructField("last_review", T.DateType(), True),
    T.StructField("review_scores_rating", T.DoubleType(), True),
    T.StructField("review_scores_accuracy", T.DoubleType(), True),
    T.StructField("review_scores_cleanliness", T.DoubleType(), True),
    T.StructField("review_scores_checkin", T.DoubleType(), True),
    T.StructField("review_scores_communication", T.DoubleType(), True),
    T.StructField("review_scores_location", T.DoubleType(), True),
    T.StructField("review_scores_value", T.DoubleType(), True),
    T.StructField("license", T.StringType(), True),
    T.StructField("instant_bookable", T.StringType(), True),
    T.StructField("calculated_host_listings_count", T.IntegerType(), True),
    T.StructField("calculated_host_listings_count_entire_homes", T.IntegerType(), True),
    T.StructField("calculated_host_listings_count_private_rooms", T.IntegerType(), True),
    T.StructField("calculated_host_listings_count_shared_rooms", T.IntegerType(), True),
    T.StructField("reviews_per_month", T.DoubleType(), True),
])


austin_listing_df=spark.read.format("csv")\
.schema(customSchema)\
.option("header", "true")\
.option("quote", "\"")\
.option("escape", "\"")\
.option("ignoreTrailingWhiteSpace", True)\
.option("multiLine", "true").load('AUSTIN/austin_listings.csv')
austin_listing_df.show(5)

+----+--------------------+--------------+------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+---------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+--

In [ ]:
austin_listing_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: decimal(20,0) (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: double (nullable = true)
 |--

In [ ]:
austin_listing_df.dtypes

[('id', 'int'),
 ('listing_url', 'string'),
 ('scrape_id', 'decimal(20,0)'),
 ('last_scraped', 'date'),
 ('name', 'string'),
 ('description', 'string'),
 ('neighborhood_overview', 'string'),
 ('picture_url', 'string'),
 ('host_id', 'int'),
 ('host_url', 'string'),
 ('host_name', 'string'),
 ('host_since', 'date'),
 ('host_location', 'string'),
 ('host_about', 'string'),
 ('host_response_time', 'string'),
 ('host_response_rate', 'string'),
 ('host_acceptance_rate', 'string'),
 ('host_is_superhost', 'string'),
 ('host_thumbnail_url', 'string'),
 ('host_picture_url', 'string'),
 ('host_neighbourhood', 'string'),
 ('host_listings_count', 'double'),
 ('host_total_listings_count', 'double'),
 ('host_verifications', 'string'),
 ('host_has_profile_pic', 'string'),
 ('host_identity_verified', 'string'),
 ('neighbourhood', 'string'),
 ('neighbourhood_cleansed', 'int'),
 ('neighbourhood_group_cleansed', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('property_type', 'string'),
 (

In [ ]:
austin_listing_df.show(5)

+----+--------------------+--------------+------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+---------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------------+--

In [ ]:
cleaned_list_columns = ['id', 'host_id', 'host_url', 'host_name', 'host_since', 
                        'host_location', 'host_is_superhost', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
                         'host_verifications', 'host_identity_verified', 'host_has_profile_pic', 'host_total_listings_count',
                        'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms_text', 
                        'bedrooms', 'beds', 'amenities', 'minimum_nights', 'maximum_nights',
                        'number_of_reviews', 'review_scores_rating', 'reviews_per_month',  ]

In [ ]:
host_columns = ['id', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified', 'host_has_profile_pic', 'host_total_listings_count']
austin_host_df = austin_listing_df.select('id', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified', 'host_has_profile_pic', 'host_total_listings_count')
austin_host_df.show(5)

+----+-------+--------------------+---------+----------+--------------------+------------------+------------------+--------------------+-----------------+----------------------+--------------------+-------------------------+
|  id|host_id|            host_url|host_name|host_since|       host_location|host_response_time|host_response_rate|host_acceptance_rate|host_is_superhost|host_identity_verified|host_has_profile_pic|host_total_listings_count|
+----+-------+--------------------+---------+----------+--------------------+------------------+------------------+--------------------+-----------------+----------------------+--------------------+-------------------------+
|2265|   2466|https://www.airbn...|    Paddy|2008-08-23|Austin, Texas, Un...|within a few hours|              100%|                 90%|                f|                     t|                   t|                      3.0|
|5245|   2466|https://www.airbn...|    Paddy|2008-08-23|Austin, Texas, Un...|within a few hours|    

In [ ]:
host_columns = ['id', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified', 'host_has_profile_pic', 'host_total_listings_count']
austin_host_df = austin_listing_df.select(*host_columns)
austin_host_df.show(5)

+----+-------+--------------------+---------+----------+--------------------+------------------+------------------+--------------------+-----------------+----------------------+--------------------+-------------------------+
|  id|host_id|            host_url|host_name|host_since|       host_location|host_response_time|host_response_rate|host_acceptance_rate|host_is_superhost|host_identity_verified|host_has_profile_pic|host_total_listings_count|
+----+-------+--------------------+---------+----------+--------------------+------------------+------------------+--------------------+-----------------+----------------------+--------------------+-------------------------+
|2265|   2466|https://www.airbn...|    Paddy|2008-08-23|Austin, Texas, Un...|within a few hours|              100%|                 90%|                f|                     t|                   t|                      3.0|
|5245|   2466|https://www.airbn...|    Paddy|2008-08-23|Austin, Texas, Un...|within a few hours|    

In [ ]:
la_listing_df=spark.read.format("csv")\
.schema(customSchema)\
.option("header", "true")\
.option("quote", "\"")\
.option("escape", "\"")\
.option("ignoreTrailingWhiteSpace", True)\
.option("ignoreLeadingWhiteSpace",True)\
.option("STOP_AT_CLOSING_QUOTE", True)\
.option("multiLine", "true").load('LA/la_listings.csv').withColumn("host_about", translate(translate('host_about',"\r",""),"\n", ","))

la_listing_df.show(5)

+----+--------------------+--------------+------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+--------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+----------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+--------------

In [ ]:
#la_listing_df = la_listing_df.withColumn("host_about", translate(translate('host_about',"\r",""),"\n", ","))
#la_listing_df.show(5)

In [ ]:
austin_listing_cleaned_df = austin_listing_df.select(*cleaned_list_columns)
la_listing_cleaned_df = la_listing_df.select(*cleaned_list_columns)

In [ ]:
austin_listing_cleaned_df = austin_listing_cleaned_df.na.drop(how='all').withColumn('property_area', lit('Texas')).withColumnRenamed('neighbourhood_cleansed', 'neighbourhood_pin').withColumnRenamed('id', 'listing_id')
la_listing_cleaned_df = la_listing_cleaned_df.na.drop(how='all').withColumn('property_area', lit('California')).withColumnRenamed('neighbourhood_cleansed', 'neighbourhood_pin').withColumnRenamed('id', 'listing_id')

In [ ]:
aust_la_listing_df_cleaned.dtypes

[('listing_id', 'int'),
 ('host_id', 'int'),
 ('host_url', 'string'),
 ('host_name', 'string'),
 ('host_since', 'date'),
 ('host_location', 'string'),
 ('host_is_superhost', 'string'),
 ('host_response_time', 'string'),
 ('host_response_rate', 'string'),
 ('host_acceptance_rate', 'string'),
 ('host_verifications', 'string'),
 ('host_identity_verified', 'string'),
 ('host_has_profile_pic', 'string'),
 ('host_total_listings_count', 'double'),
 ('neighbourhood_pin', 'int'),
 ('property_type', 'string'),
 ('room_type', 'string'),
 ('accommodates', 'int'),
 ('bathrooms_text', 'string'),
 ('bedrooms', 'double'),
 ('beds', 'double'),
 ('amenities', 'string'),
 ('minimum_nights', 'int'),
 ('maximum_nights', 'int'),
 ('number_of_reviews', 'int'),
 ('review_scores_rating', 'double'),
 ('reviews_per_month', 'double'),
 ('property_area', 'string')]

In [ ]:
aust_la_listing_df_cleaned = austin_listing_cleaned_df.union(la_listing_cleaned_df)
aust_la_listing_df_cleaned.show()

+----------+-------+--------------------+---------+----------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+----------------------+--------------------+-------------------------+-----------------+--------------------+---------------+------------+--------------+--------+----+--------------------+--------------+--------------+-----------------+--------------------+-----------------+-------------+
|listing_id|host_id|            host_url|host_name|host_since|       host_location|host_is_superhost|host_response_time|host_response_rate|host_acceptance_rate|  host_verifications|host_identity_verified|host_has_profile_pic|host_total_listings_count|neighbourhood_pin|       property_type|      room_type|accommodates|bathrooms_text|bedrooms|beds|           amenities|minimum_nights|maximum_nights|number_of_reviews|review_scores_rating|reviews_per_month|property_area|
+----------+-------+--------------------+---------+-------

In [ ]:
aust_la_listing_df_cleaned.count()

42172

In [ ]:
aust_la_listing_df_cleaned.distinct().count()

42172

In [ ]:
aust_la_listing_df_cleaned.coalesce(1).write.format('csv').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/airbnb_listing_austin_la_cleaned_output.csv')

In [ ]:
dfcsv = spark.read.csv("/content/drive/MyDrive/Udacity Data Engineering/austin_la_output_files_final/airbnb_listing_austin_la_cleaned_output.csv")
dfcsv.show()

+----------+-------+--------------------+---------+----------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+---------------+------------+--------------+--------+----+--------------------+--------------+--------------+-----------------+--------------------+-----------------+-------------+
|       _c0|    _c1|                 _c2|      _c3|       _c4|                 _c5|              _c6|               _c7|               _c8|                 _c9|                _c10|                _c11|                _c12|                _c13|             _c14|                _c15|           _c16|        _c17|          _c18|    _c19|_c20|                _c21|          _c22|          _c23|             _c24|                _c25|             _c26|         _c27|
+----------+-------+--------------------+---------+----------+----------

In [ ]:
aust_la_listing_df_cleaned\
            .coalesce(1)\
            .write\
            .mode('overwrite')\
            .csv('/content/drive/MyDrive/Udacity Data Engineering/listing_austin_la_cleaned_output.csv', header = 'true')

In [ ]:
f = "/content/drive/MyDrive/Udacity Data Engineering/austin_la_output_files_final/airbnb_listing_austin_la_cleaned_output.csv"


dfcsv1 = spark.read.format('csv').options(header='true').load(f)
                  
dfcsv1.show()

+----------+-------+--------------------+---------+----------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+----------------------+--------------------+-------------------------+-----------------+--------------------+---------------+------------+--------------+--------+----+--------------------+--------------+--------------+-----------------+--------------------+-----------------+-------------+
|listing_id|host_id|            host_url|host_name|host_since|       host_location|host_is_superhost|host_response_time|host_response_rate|host_acceptance_rate|  host_verifications|host_identity_verified|host_has_profile_pic|host_total_listings_count|neighbourhood_pin|       property_type|      room_type|accommodates|bathrooms_text|bedrooms|beds|           amenities|minimum_nights|maximum_nights|number_of_reviews|review_scores_rating|reviews_per_month|property_area|
+----------+-------+--------------------+---------+-------

In [ ]:
df2 = spark.read.parquet("/content/drive/MyDrive/Udacity Data Engineering/listing_austin_la_cleaned_output.csv/part-00000-a6046ebe-a494-4429-bcfa-6526d09fa429-c000.snappy.parquet")
df2.show()

+----------+-------+--------------------+---------+----------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+----------------------+--------------------+-------------------------+-----------------+--------------------+---------------+------------+--------------+--------+----+--------------------+--------------+--------------+-----------------+--------------------+-----------------+-------------+
|listing_id|host_id|            host_url|host_name|host_since|       host_location|host_is_superhost|host_response_time|host_response_rate|host_acceptance_rate|  host_verifications|host_identity_verified|host_has_profile_pic|host_total_listings_count|neighbourhood_pin|       property_type|      room_type|accommodates|bathrooms_text|bedrooms|beds|           amenities|minimum_nights|maximum_nights|number_of_reviews|review_scores_rating|reviews_per_month|property_area|
+----------+-------+--------------------+---------+-------

### CALENDER TABLE

In [ ]:
cal_customSchema = StructType([
    T.StructField("listing_id", T.IntegerType(), True),
    T.StructField("date", T.DateType(), True),
    T.StructField("available", T.StringType(), True),
    T.StructField("price", T.StringType(), True),
    T.StructField("adjusted_price", T.StringType(), True),
    T.StructField("minimum_nights", T.IntegerType(), True),
    T.StructField("maximum_nights", T.IntegerType(), True),
])


aust_cal_df = spark.read.format("csv").schema(cal_customSchema).option("header", "true").load('AUSTIN/austin_calender.csv')

#aust_cal_df = spark.read.csv('/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/austin_calender.csv', header=True)
aust_cal_df = aust_cal_df.withColumn('price', translate('price', '$', '')).withColumn('price', col('price').cast('float'))\
                          .withColumn('adjusted_price', translate('adjusted_price', '$', '')).withColumn('adjusted_price', col('adjusted_price').cast('float'))
#aust_cal_df = aust_cal_df.withColumn('adjusted_price', translate('adjusted_price', '$', '')).withColumn('adjusted_price', col('adjusted_price').cast('float'))
aust_cal_df.show(5)


+----------+----------+---------+-----+--------------+--------------+--------------+
|listing_id|      date|available|price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-----+--------------+--------------+--------------+
|     77635|2021-02-12|        f|175.0|         175.0|             2|          1125|
|     76925|2021-02-12|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-13|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-14|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-15|        f|400.0|         400.0|             2|             8|
+----------+----------+---------+-----+--------------+--------------+--------------+
only showing top 5 rows



In [ ]:
la_cal_df = spark.read.format("csv").schema(cal_customSchema).option("header", "true").load('LA/la_calendar.csv')
la_cal_df = la_cal_df.withColumn('price', translate('price', '$', '')).withColumn('price', col('price').cast('float'))
la_cal_df = la_cal_df.withColumn('adjusted_price', translate('adjusted_price', '$', '')).withColumn('adjusted_price', col('adjusted_price').cast('float'))
la_cal_df.show(5)

+----------+----------+---------+-----+--------------+--------------+--------------+
|listing_id|      date|available|price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-----+--------------+--------------+--------------+
|     85246|2021-02-09|        t| 95.0|          95.0|            30|           190|
|       109|2021-02-09|        f|115.0|         115.0|            30|           730|
|       109|2021-02-10|        f|115.0|         115.0|            30|           730|
|       109|2021-02-11|        f|115.0|         115.0|            30|           730|
|       109|2021-02-12|        f|115.0|         115.0|            30|           730|
+----------+----------+---------+-----+--------------+--------------+--------------+
only showing top 5 rows



In [ ]:
la_cal_df_cleaned = la_cal_df.na.drop(how='all').where(col('listing_id').isNotNull())
aust_cal_df_cleaned = aust_cal_df.na.drop(how='all').where(col('listing_id').isNotNull())

In [ ]:
la_cal_df.schema

StructType(List(StructField(listing_id,IntegerType,true),StructField(date,DateType,true),StructField(available,StringType,true),StructField(price,FloatType,true),StructField(adjusted_price,FloatType,true),StructField(minimum_nights,IntegerType,true),StructField(maximum_nights,IntegerType,true)))

In [ ]:
aust_la_cal_df_cleaned = aust_cal_df_cleaned.union(la_cal_df_cleaned)
aust_la_cal_df_cleaned.show()

+----------+----------+---------+-----+--------------+--------------+--------------+
|listing_id|      date|available|price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-----+--------------+--------------+--------------+
|     77635|2021-02-12|        f|175.0|         175.0|             2|          1125|
|     76925|2021-02-12|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-13|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-14|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-15|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-16|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-17|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-18|        f|400.0|         400.0|             2|             8|
|     76925|2021-02-19|        f|400.0|         400.0|           

In [ ]:
aust_la_cal_df_cleaned.printSchema()

root
 |-- listing_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- available: string (nullable = true)
 |-- price: float (nullable = true)
 |-- adjusted_price: float (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- maximum_nights: integer (nullable = true)



In [ ]:
aust_la_cal_df_cleaned.count()

13314612

In [ ]:
write_folder = '/content/drive/MyDrive/Udacity Data Engineering/airbnb_calender_austin_la'
aust_la_cal_df_cleaned.coalesce(1).write.option("header", "true").mode("overwrite").parquet(write_folder)

In [ ]:
aust_la_cal_df_cleaned.coalesce(1).write.format('csv').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/airbnb_calender_austin_la_cleaned_output.csv')

### REVIEW TABLE

In [ ]:
#### 

In [ ]:
rev_customSchema = StructType([
    T.StructField("listing_id", T.IntegerType(), True),
    T.StructField("id", T.DecimalType(20), True),
    T.StructField("date", T.DateType(), True),
    T.StructField("reviewer_id", T.DecimalType(20), True),
    T.StructField("reviewer_name", T.StringType(), True),
    T.StructField("comments", T.StringType(), True),
])

aust_review_df = spark.read.format("csv").schema(rev_customSchema).option("header", "true").load('AUSTIN/austin_reviews.csv')
#aust_review_df = spark.read.csv('/content/drive/MyDrive/Udacity Data Engineering/airbnb/AUSTIN/austin_reviews.csv', header=True)
aust_review_df.show(5)

+----------+-------+----------+-----------+-------------+--------------------+
|listing_id|     id|      date|reviewer_id|reviewer_name|            comments|
+----------+-------+----------+-----------+-------------+--------------------+
|      2265|    963|2009-03-17|       7538|        Niall|I stayed here dur...|
|      2265|   1057|2009-03-22|      10029|      Michael|Great place, clos...|
|      2265| 200418|2011-03-16|      61677|       Gustaf|We had a great ti...|
|      2265|1001630|2012-03-15|    1523753|         Noah|We had a great st...|
|      2265|1016390|2012-03-19|    1547660|      Melissa|I arrived late in...|
+----------+-------+----------+-----------+-------------+--------------------+
only showing top 5 rows



In [ ]:
la_review_df = spark.read.format("csv").schema(rev_customSchema)\
.option("quote", "\"")\
.option("escape", "\"")\
.option("ignoreTrailingWhiteSpace", True)\
.option("ignoreLeadingWhiteSpace",True)\
.option("header", "true").load('LA/la_reviews.csv')
la_review_df.show(5)

+----------+--------+----------+-----------+-------------+--------------------+
|listing_id|      id|      date|reviewer_id|reviewer_name|            comments|
+----------+--------+----------+-----------+-------------+--------------------+
|       109|  449036|2011-08-15|     927861|        Edwin|The host canceled...|
|       109|74506539|2016-05-15|   22509885|         Jenn|Me and two friend...|
|      2708|13994902|2014-06-09|   10905424|      Kuberan|i had a wonderful...|
|      null|    null|      null|       null|         null|                null|
|      2708|14606598|2014-06-23|    2247288|      Camilla|Charles is just a...|
+----------+--------+----------+-----------+-------------+--------------------+
only showing top 5 rows



In [ ]:
la_review_df_cleaned = la_review_df.na.drop(how='all').where(col('listing_id').isNotNull()).withColumn('Area', lit('California'))

aust_review_df_cleaned = aust_review_df.na.drop(how='all').where(col('listing_id').isNotNull()).withColumn('Area', lit('Texas'))

In [ ]:
la_review_df_cleaned.show()

+----------+---------+----------+-----------+----------------+----------------------------------+----------+
|listing_id|       id|      date|reviewer_id|   reviewer_name|                          comments|      Area|
+----------+---------+----------+-----------+----------------+----------------------------------+----------+
|       109|   449036|2011-08-15|     927861|           Edwin|              The host canceled...|California|
|       109| 74506539|2016-05-15|   22509885|            Jenn|              Me and two friend...|California|
|      2708| 13994902|2014-06-09|   10905424|         Kuberan|              i had a wonderful...|California|
|      2708| 14606598|2014-06-23|    2247288|         Camilla|              Charles is just a...|California|
|      2708| 39597339|2015-07-25|   27974696|          Fallon|              Staying with Chas...|California|
|      2708| 61157407|2016-02-01|   33226412|          Haroon|              Charles is a most...|California|
|      2708| 661962

In [ ]:
aust_la_review_df_cleaned = aust_review_df_cleaned.union(la_review_df_cleaned)

In [ ]:
aust_la_review_df_cleaned.printSchema()

root
 |-- listing_id: integer (nullable = true)
 |-- id: decimal(20,0) (nullable = true)
 |-- date: date (nullable = true)
 |-- reviewer_id: decimal(20,0) (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- Area: string (nullable = false)



In [ ]:
aust_la_review_df_cleaned.show()

+----------+---------+----------+-----------+-------------+--------------------+-----+
|listing_id|       id|      date|reviewer_id|reviewer_name|            comments| Area|
+----------+---------+----------+-----------+-------------+--------------------+-----+
|      2265|      963|2009-03-17|       7538|        Niall|I stayed here dur...|Texas|
|      2265|     1057|2009-03-22|      10029|      Michael|Great place, clos...|Texas|
|      2265|   200418|2011-03-16|      61677|       Gustaf|We had a great ti...|Texas|
|      2265|  1001630|2012-03-15|    1523753|         Noah|We had a great st...|Texas|
|      2265|  1016390|2012-03-19|    1547660|      Melissa|I arrived late in...|Texas|
|      2265|  2707758|2012-10-24|    2449913|         Nick|a really welcomin...|Texas|
|      2265|  2926194|2012-11-20|    3362054|      Chrysta|We stayed here fo...|Texas|
|      2265|  3813128|2013-03-18|    4025558|         John|Paddy's place was...|Texas|
|      2265| 11005865|2014-03-17|   1160496

In [ ]:
aust_la_review_df_cleaned = aust_la_review_df_cleaned.withColumnRenamed('id', 'review_id').withColumnRenamed('date', 'review_date')
aust_la_review_df_cleaned.show()

+----------+---------+-----------+-----------+-------------+--------------------+-----+
|listing_id|review_id|review_date|reviewer_id|reviewer_name|            comments| Area|
+----------+---------+-----------+-----------+-------------+--------------------+-----+
|      2265|      963| 2009-03-17|       7538|        Niall|I stayed here dur...|Texas|
|      2265|     1057| 2009-03-22|      10029|      Michael|Great place, clos...|Texas|
|      2265|   200418| 2011-03-16|      61677|       Gustaf|We had a great ti...|Texas|
|      2265|  1001630| 2012-03-15|    1523753|         Noah|We had a great st...|Texas|
|      2265|  1016390| 2012-03-19|    1547660|      Melissa|I arrived late in...|Texas|
|      2265|  2707758| 2012-10-24|    2449913|         Nick|a really welcomin...|Texas|
|      2265|  2926194| 2012-11-20|    3362054|      Chrysta|We stayed here fo...|Texas|
|      2265|  3813128| 2013-03-18|    4025558|         John|Paddy's place was...|Texas|
|      2265| 11005865| 2014-03-1

In [ ]:
aust_la_review_df_cleaned.count()

1395707

In [ ]:
aust_la_review_df_cleaned.coalesce(1).write.format('parquet').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/airbnb_reviews_austin_la.parquet')

In [ ]:
write_folder = '/content/drive/MyDrive/Udacity Data Engineering/airbnb_reviews_austin_la_pp'
aust_la_review_df_cleaned.coalesce(1).write.option("header", "true").mode("overwrite").parquet(write_folder)

In [ ]:
aust_la_review_df_cleaned.coalesce(1).write.format('json').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/airbnb_reviews_austin_la_cleaned_output.json')

In [ ]:
aust_la_review_df_cleaned.coalesce(1).write.format('csv').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/airbnb_reviews_austin_la_cleaned_output.csv')

In [ ]:
#aust_la_review_df_cleaned.write.format('csv').option('header',True).mode('overwrite').option('sep','|').save('/content/drive/MyDrive/Udacity Data Engineering/austin_la_review_cleaned_output.csv')

In [ ]:
aust_la_review_df_cleaned.coalesce(1).write.format('csv').option('header',True).mode('overwrite').save('/content/drive/MyDrive/Udacity Data Engineering/austin_la_review_cleaned_output.csv')

In [ ]:
csv_la_austin_review = '/content/drive/MyDrive/Udacity Data Engineering/austin_la_review_cleaned_output.csv/part-00000-09c120eb-5f32-48f3-bd0d-4a4c44704222-c000.csv'
json_la_austin_review = '/content/drive/MyDrive/Udacity Data Engineering/airbnb/new_review_cleaned.csv/austin_la_review.json'


make_json(csv_la_austin_review, json_la_austin_review)

### Rough

In [ ]:
aust_la_cal_df_cleaned
aust_la_listing_df_cleaned
aust_la_review_df_cleaned

aust_la_review_df_cleaned.createOrReplaceTempView("temp")

In [ ]:
aust_la_listing_df_cleaned.createOrReplaceTempView("stg_listing")
#spark.sql("  FROM stg_listing ").show()

In [ ]:
spark.sql("SELECT DISTINCT HOST_ID, HOST_NAME, HOST_URL, \
HOST_SINCE, HOST_IDENTITY_VERIFIED, HOST_IS_SUPERHOST, HOST_RESPONSE_TIME, HOST_RESPONSE_RATE, HOST_ACCEPTANCE_RATE, \
HOST_VERIFICATIONS, HOST_HAS_PROFILE_PIC, HOST_TOTAL_LISTINGS_COUNT \
FROM stg_listing \
where HOST_NAME <> 'Not Provided' AND HOST_SINCE <> '2000-01-01'").createOrReplaceTempView("stg_hosts_sql")


In [ ]:
aust_la_listing_df_cleaned.select([max("host_total_listings_count")]).show()
#row1.show()

+------------------------------+
|max(host_total_listings_count)|
+------------------------------+
|                        7421.0|
+------------------------------+



In [ ]:
aust_la_listing_df_cleaned.filter(aust_la_listing_df_cleaned['host_total_listings_count'])

In [ ]:
spark.sql("SELECT COUNT(*) FROM stg_listing")
#spark.sql("SELECT * FROM stg_listing order by host_total_listings_count desc").show()

count(1)
42172


In [ ]:
spark.sql("SELECT COUNT(*) FROM stg_hosts_sql")

count(1)
3444


In [ ]:
spark.sql("Select count(*) from stg_properties")

count(1)
42172


In [ ]:
spark.sql("SELECT DISTINCT LISTING_ID, HOST_ID, PROPERTY_TYPE, \
ROOM_TYPE, ACCOMMODATES, BEDROOMS, BEDS, BATHROOMS_TEXT, \
AMENITIES, NEIGHBOURHOOD_PIN, property_area \
FROM stg_listing ").createOrReplaceTempView("stg_properties")

spark.sql("Select * from stg_properties").show()

+----------+--------+--------------------+---------------+------------+--------+----+--------------+--------------------+-----------------+-------------+
|LISTING_ID| HOST_ID|       PROPERTY_TYPE|      ROOM_TYPE|ACCOMMODATES|BEDROOMS|BEDS|BATHROOMS_TEXT|           AMENITIES|NEIGHBOURHOOD_PIN|property_area|
+----------+--------+--------------------+---------------+------------+--------+----+--------------+--------------------+-----------------+-------------+
|     72876|  379080|Private room in h...|   Private room|           2|    null| 0.0|0 shared baths|["Hair dryer", "S...|            78729|        Texas|
|     83643|  456230|Private room in h...|   Private room|           2|     1.0| 1.0| 1 shared bath|["Free parking on...|            78752|        Texas|
|    319759|  814883|    Entire townhouse|Entire home/apt|          16|     5.0| 9.0|     3.5 baths|["Hair dryer", "P...|            78702|        Texas|
|    355328| 1798834|        Entire house|Entire home/apt|           2|     

In [ ]:
aust_la_review_df_cleaned.createOrReplaceTempView("temp")

In [ ]:
spark.sql("select * from temp").show()

+----------+---------+----------+-----------+-------------+--------------------+-----+
|listing_id|       id|      date|reviewer_id|reviewer_name|            comments| Area|
+----------+---------+----------+-----------+-------------+--------------------+-----+
|      2265|      963|2009-03-17|       7538|        Niall|I stayed here dur...|Texas|
|      2265|     1057|2009-03-22|      10029|      Michael|Great place, clos...|Texas|
|      2265|   200418|2011-03-16|      61677|       Gustaf|We had a great ti...|Texas|
|      2265|  1001630|2012-03-15|    1523753|         Noah|We had a great st...|Texas|
|      2265|  1016390|2012-03-19|    1547660|      Melissa|I arrived late in...|Texas|
|      2265|  2707758|2012-10-24|    2449913|         Nick|a really welcomin...|Texas|
|      2265|  2926194|2012-11-20|    3362054|      Chrysta|We stayed here fo...|Texas|
|      2265|  3813128|2013-03-18|    4025558|         John|Paddy's place was...|Texas|
|      2265| 11005865|2014-03-17|   1160496

In [ ]:
spark.sql("select count(*) from temp").show()

+--------+
|count(1)|
+--------+
| 1395707|
+--------+

